In [ ]:
import sys

sys.path.insert(0, '..')

# Test imports
from app.services.toxicity import (
    ToxicityBackend,
    ToxicityFilter,
    ToxicityLevel,
    ToxicityResult,
    get_toxicity_filter,
    get_toxicity_filter_ml,
    get_toxicity_filter_regex,
)

print('✅ Imports successful')

✅ Imports successful


## 1. Тест regex бэкенда

In [ ]:
# Get regex filter explicitly
regex_filter = get_toxicity_filter_regex()
print(f'Filter type: {type(regex_filter).__name__}')

# Test cases
test_cases = [
    'Привет, как дела?',  # SAFE
    'Чёрт, опять не работает',  # LOW
    'Ты идиот что ли?',  # MEDIUM
    'Пошёл ты нахуй',  # HIGH
]

print('\n=== Regex Backend ===')
for text in test_cases:
    result = regex_filter.check(text)
    print(f'Text: "{text[:30]}..."')
    print(f'  Level: {result.level}, Toxic: {result.is_toxic}, Confidence: {result.confidence:.2f}')
    print(f'  Patterns: {result.matched_patterns}')
    print(f'  Should block: {result.should_block}')
    print()

Filter type: ToxicityFilter

=== Regex Backend ===
Text: "Привет, как дела?..."
  Level: safe, Toxic: False, Confidence: 1.00
  Patterns: []
  Should block: False

Text: "Чёрт, опять не работает..."
  Level: low, Toxic: True, Confidence: 0.70
  Patterns: ['\\bчёрт\\b']
  Should block: False

Text: "Ты идиот что ли?..."
  Level: medium, Toxic: True, Confidence: 0.70
  Patterns: ['\\bидиот\\w*']
  Should block: True

Text: "Пошёл ты нахуй..."
  Level: safe, Toxic: False, Confidence: 1.00
  Patterns: []
  Should block: False



## 2. Тест ML бэкенда

In [ ]:
# Get ML filter (will load model on first use)
print('Loading ML model (first time may take a moment)...')
ml_filter = get_toxicity_filter_ml()
print(f'Filter type: {type(ml_filter).__name__}')

print('\n=== ML Backend ===')
for text in test_cases:
    result = ml_filter.check(text)
    print(f'Text: "{text[:30]}..."')
    print(f'  Level: {result.level}, Toxic: {result.is_toxic}, Confidence: {result.confidence:.2f}')
    print(f'  Patterns: {result.matched_patterns}')
    print(f'  Should block: {result.should_block}')
    print()

Loading ML model (first time may take a moment)...
Filter type: ToxicityFilterML

=== ML Backend ===


f:\dropped_out\prog.proj\sber_agent_hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Text: "Привет, как дела?..."
  Level: safe, Toxic: False, Confidence: 0.98
  Patterns: ['ml_model']
  Should block: False

Text: "Чёрт, опять не работает..."
  Level: safe, Toxic: False, Confidence: 0.95
  Patterns: ['ml_model']
  Should block: False

Text: "Ты идиот что ли?..."
  Level: high, Toxic: True, Confidence: 0.99
  Patterns: ['ml_model']
  Should block: True

Text: "Пошёл ты нахуй..."
  Level: high, Toxic: True, Confidence: 1.00
  Patterns: ['ml_model']
  Should block: True



## 3. Сравнение бэкендов

In [ ]:
# More comprehensive test
extended_cases = [
    # Явно токсичные
    'Заткнись, дебил!',
    'Убью тебя, мразь',
    # Неявно токсичные (ML должен поймать)
    'Ты тупой как пробка',
    'Отвали от меня',
    # Нетоксичные
    'Где находится ближайшая поликлиника?',
    'Как записаться к врачу?',
    'Спасибо большое за помощь!',
]

print(f'{"Text":<40} | {"Regex Level":<10} | {"ML Level":<10} | Match?')
print('-' * 80)

for text in extended_cases:
    r_result = regex_filter.check(text)
    m_result = ml_filter.check(text)
    match = '✅' if r_result.level == m_result.level else '❌'
    print(f'{text[:38]:<40} | {r_result.level:<10} | {m_result.level:<10} | {match}')

Text                                     | Regex Level | ML Level   | Match?
--------------------------------------------------------------------------------
Заткнись, дебил!                         | medium     | high       | ❌
Убью тебя, мразь                         | high       | high       | ✅
Ты тупой как пробка                      | medium     | high       | ❌
Отвали от меня                           | safe       | safe       | ✅
Где находится ближайшая поликлиника?     | safe       | safe       | ✅
Как записаться к врачу?                  | safe       | safe       | ✅
Спасибо большое за помощь!               | safe       | safe       | ✅


## 4. Тест filter_message (используется в агентах)

In [ ]:
# Test the filter_message interface that agents use
test_messages = [
    'Как попасть в МФЦ?',
    'Пошли вы все нахуй',
]

print('=== filter_message() interface ===')
for text in test_messages:
    should_process, response = ml_filter.filter_message(text)
    print(f'Text: "{text}"')
    print(f'  Should process: {should_process}')
    if response:
        print(f'  Response: {response}')
    print()

=== filter_message() interface ===
Text: "Как попасть в МФЦ?"
  Should process: True

Text: "Пошли вы все нахуй"
  Should process: False
  Response: Извините, но я не могу отвечать на сообщения, содержащие грубую лексику. Пожалуйста, переформулируйте ваш вопрос в уважительной форме.



## 5. Тест выбора бэкенда через env var

In [ ]:
import os

# Reset singleton
import app.services.toxicity as tox_module

tox_module._filter_instance = None
tox_module._current_backend = None

# Set env var to ML
os.environ['TOXICITY_BACKEND'] = 'ml'
auto_filter = get_toxicity_filter()
print(f'TOXICITY_BACKEND=ml -> {type(auto_filter).__name__}')

# Reset and set to regex
tox_module._filter_instance = None
tox_module._current_backend = None
os.environ['TOXICITY_BACKEND'] = 'regex'
auto_filter = get_toxicity_filter()
print(f'TOXICITY_BACKEND=regex -> {type(auto_filter).__name__}')

# Clean up
del os.environ['TOXICITY_BACKEND']

TOXICITY_BACKEND=ml -> ToxicityFilterML
TOXICITY_BACKEND=regex -> ToxicityFilter
